<a href="https://colab.research.google.com/github/lisa11323/CSR_Laddit/blob/studay2/study_2_network_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Network Metrics Calculation: Density, Average Clustering Coefficient, Reciprocity, Vertical vs Horizontal Relationships

In [ ]:
# ✅ 라이브러리 설치
!pip install networkx tqdm

import pandas as pd
import networkx as nx
from tqdm import tqdm
from google.colab import files

# ✅ 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# ✅ 데이터 로드
df = pd.read_csv(filename)

print(f"데이터 로드 완료: {filename}")
print(df.head())

# ✅ parent_author 생성
df['parent_comment_id'] = df['parent_id'].str.replace('t1_', '', regex=False).str.replace('t3_', '', regex=False)
parent_authors = df.set_index('comment_id')['author'].to_dict()
df['parent_author'] = df['parent_comment_id'].map(parent_authors)

results = []

for post_id, group in tqdm(df.groupby('post_id'), desc='게시물별 네트워크 분석'):
    G = nx.DiGraph()

    # 노드 및 엣지 추가
    for _, row in group.iterrows():
        G.add_node(row['author'], created_utc=row['created_utc'])

    for _, row in group.iterrows():
        if pd.notnull(row['parent_author']) and row['parent_author'] != '[deleted]':
            G.add_edge(row['author'], row['parent_author'], created_utc=row['created_utc'])

    # 지표 계산
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    density = nx.density(G)
    reciprocity = nx.reciprocity(G) if num_edges > 1 else 0
    avg_clustering = nx.average_clustering(G.to_undirected()) if num_edges > 1 else 0

    # 수평/수직성 분석
    degrees = dict(G.in_degree())
    if degrees:
        max_central_node, max_central_degree = max(degrees.items(), key=lambda x: x[1])
        star_ratio = max_central_degree / num_edges if num_edges > 0 else 0
    else:
        star_ratio = 0

    results.append({
        'post_id': post_id,
        'num_nodes': num_nodes,
        'num_edges': num_edges,
        'density': density,
        'reciprocity': reciprocity,
        'avg_clustering': avg_clustering,
        'star_ratio': star_ratio  # 1에 가까울수록 수직(별 구조), 낮을수록 수평
    })

df_results = pd.DataFrame(results)

output_filename = "network_metrics_with_star_ratio.csv"
df_results.to_csv(output_filename, index=False)

print(f"✅ 분석 결과 저장됨: {output_filename}")

# ✅ 파일 다운로드
files.download(output_filename)

Saving reddit_comments_crawled.csv to reddit_comments_crawled (1).csv
데이터 로드 완료: reddit_comments_crawled (1).csv
  post_id comment_id  parent_id        author  \
0  wljav4    imjqvxr  t3_wljav4     elderrion   
1  wljav4    inp2x4i  t3_wljav4  Ginja4Ninjaa   
2  wljav4    iqandhn  t3_wljav4      Keats852   
3  wljav4    ivnqiwf  t3_wljav4           NaN   
4  wljav4    ilhzxgm  t3_wljav4           NaN   

                                                body   created_utc  
0  She asked me if I knew any good books or podca...  1.661967e+09  
1  I think the prices of items are outrageously h...  1.662706e+09  
2  How do I get my Replika to take the lead on co...  1.664404e+09  
3                                          [deleted]  1.667981e+09  
4  Is it anyway to turn off the random meme the r...  1.661285e+09  


게시물별 네트워크 분석: 100%|██████████| 1043/1043 [00:03<00:00, 274.19it/s]


✅ 분석 결과 저장됨: network_metrics_with_star_ratio.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2-1.Exclude Zero Values

In [ ]:
import pandas as pd
from google.colab import files

# 📌 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df_metrics = pd.read_csv(filename)

# 📌 개선된 선정 기준
df_selected = df_metrics[
    (df_metrics['num_nodes'] >= 10) &
    (df_metrics['density'] > 0) &
    (df_metrics['reciprocity'] > 0) &
    (df_metrics['avg_clustering'] > 0) &
    (df_metrics['star_ratio'] > 0)
]

print(f"선정된 게시물 수: {len(df_selected)}")

df_selected.to_csv("selected_posts.csv", index=False)

# 📌 다운로드
files.download("selected_posts.csv")

Saving network_metrics_with_star_ratio.csv to network_metrics_with_star_ratio.csv
선정된 게시물 수: 137


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2-2. Select Posts for Analysis

In [ ]:
# 📌 라이브러리
import pandas as pd
from google.colab import files

# 📌 파일 업로드
print("🔷 'selected_posts.csv' 파일을 업로드하세요.")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

print(f"✅ 파일 로드 완료: {filename}")
print(f"총 게시물 수: {len(df)}")

# 📌 선정 기준 적용
print("🔷 기준별 상위 게시물 선정 중…")

# density 상위 10
top_density = df.sort_values('density', ascending=False).head(10)

# reciprocity 상위 10
top_reciprocity = df.sort_values('reciprocity', ascending=False).head(10)

# star_ratio 상위 5 (수직)
top_star = df.sort_values('star_ratio', ascending=False).head(5)

# star_ratio 하위 5 (수평)
low_star = df.sort_values('star_ratio', ascending=True).head(5)

# 📌 합치기
final_selection = pd.concat([top_density, top_reciprocity, top_star, low_star]).drop_duplicates()

print(f"✅ 최종 선정된 게시물 수: {len(final_selection)}")

# 📌 결과 저장
output_filename = "visualization_targets.csv"
final_selection.to_csv(output_filename, index=False)

# 📌 다운로드
files.download(output_filename)

print(f"🎯 선정된 게시물 목록이 '{output_filename}'로 다운로드되었습니다.")

🔷 'selected_posts.csv' 파일을 업로드하세요.


Saving selected_posts (1).csv to selected_posts (1).csv
✅ 파일 로드 완료: selected_posts (1).csv
총 게시물 수: 137
🔷 기준별 상위 게시물 선정 중…
✅ 최종 선정된 게시물 수: 28


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎯 선정된 게시물 목록이 'visualization_targets.csv'로 다운로드되었습니다.


# 3. Detailed Analysis per Selected Post

In [ ]:
!pip install networkx matplotlib tqdm

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from tqdm import tqdm
from google.colab import files

# ✅ 파일 업로드: 크롤링 데이터
print("🔷 크롤링 데이터 파일 업로드")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

# ✅ 파일 업로드: 선정된 게시물 목록
print("🔷 시각화 대상 게시물 파일 업로드")
uploaded = files.upload()
selected_filename = list(uploaded.keys())[0]
df_selected = pd.read_csv(selected_filename)

selected_post_ids = df_selected['post_id'].tolist()
print(f"✅ 선정된 게시물 수: {len(selected_post_ids)}")

# ✅ parent_author 생성
df['parent_comment_id'] = df['parent_id'].str.replace('t1_', '', regex=False).str.replace('t3_', '', regex=False)
parent_authors = df.set_index('comment_id')['author'].to_dict()
df['parent_author'] = df['parent_comment_id'].map(parent_authors)

# ✅ PDF 파일 생성
output_pdf = "network_visualizations_with_metrics.pdf"
pdf = PdfPages(output_pdf)

for post_id in tqdm(selected_post_ids, desc="시각화 중"):
    group = df[df['post_id'] == post_id]
    G = nx.DiGraph()

    # 노드 및 엣지 추가
    for _, row in group.iterrows():
        G.add_node(row['author'], created_utc=row['created_utc'])

    for _, row in group.iterrows():
        if pd.notnull(row['parent_author']) and row['parent_author'] != '[deleted]':
            G.add_edge(row['author'], row['parent_author'], created_utc=row['created_utc'])

    # 📌 네트워크 지표 계산
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    density = nx.density(G)
    reciprocity = nx.reciprocity(G) if num_edges > 1 else 0
    avg_clustering = nx.average_clustering(G.to_undirected()) if num_edges > 1 else 0

    degrees = dict(G.in_degree())
    if degrees:
        max_central_node, max_central_degree = max(degrees.items(), key=lambda x: x[1])
        star_ratio = max_central_degree / num_edges if num_edges > 0 else 0
    else:
        star_ratio = 0

    # 📌 시각화
    plt.figure(figsize=(8, 8))
    pos = nx.spring_layout(G, seed=42)
    nx.draw_networkx_nodes(G, pos, node_size=50, node_color='skyblue')
    nx.draw_networkx_edges(G, pos, arrows=True, alpha=0.5)

    plt.title(
        f"Post ID: {post_id}\n"
        f"Nodes: {num_nodes} | Edges: {num_edges} | Density: {density:.3f} | "
        f"Reciprocity: {reciprocity:.3f} | Clustering: {avg_clustering:.3f} | "
        f"Star Ratio: {star_ratio:.3f}",
        fontsize=10
    )
    plt.axis('off')

    # 📌 현재 페이지 PDF에 저장
    pdf.savefig()
    plt.close()

pdf.close()
print(f"✅ PDF 저장 완료: {output_pdf}")

# ✅ 다운로드
files.download(output_pdf)

🔷 크롤링 데이터 파일 업로드


Saving 0.reddit_comments_crawled.csv to 0.reddit_comments_crawled.csv
🔷 시각화 대상 게시물 파일 업로드


Saving 2-2. 게시물 선정.csv to 2-2. 게시물 선정.csv
✅ 선정된 게시물 수: 28


시각화 중: 100%|██████████| 28/28 [00:06<00:00,  4.19it/s]

✅ PDF 저장 완료: network_visualizations_with_metrics.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>